<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# Run this before everything
!pip install --upgrade --q tensorflow

In [2]:
from google.colab import drive
import pandas as pd
from pandas import read_csv, to_datetime
import numpy as np
from numpy import array,hstack
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
plt.rcParams['figure.figsize'] = 10, 5
import seaborn as sns
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import datetime


In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
cd /content/gdrive/My Drive/KeyEnergy/Armidale/

/content/gdrive/My Drive/KeyEnergy/Armidale


In [ ]:
ls

2017_Armidale.csv
2018_Armidale.csv
2019_Armidale.csv
aiplatformmvp/
Armidale2019_Sim_v1_Harry25Feb.csv
Armidale_AutumnHoliday_60steps_2031test.h5
Armidale_AutumnHoliday_60steps_20.54test.h5
Armidale_AutumnHoliday_60steps_225.h5
Armidale.csv
Armidale_SpringHoliday_60steps_24.h5
Armidale_Summer_12_steps_Mul_14.57.h5
Armidale_SummerHoliday_60steps_16.h5
Armidale_SummerHoliday_60steps_1746.h5
Armidale_SummerHoliday_60steps_18.h5
Armidale_WinterHoliday_60steps_28.h5
Armidale_WinterHoliday_60steps_30.h5
LSTM_Armidale.ipynb
Parameters.xlsx
public_holiday.csv
scaler.save
summer30/
terms_date.csv


In [5]:
# Import the dataset
#drop = ['kVAR', 'kW', 'kWh Value','kVARh Value','kWh Actual', 'Max kW','Period', 'No Of Meters', 'kWh Estimate', 'TOU Demand kW', 'TOU Demand kVA', 'PF', 'CO2']
datasetAll = read_csv(r'Armidale.csv', header = 0)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(datasetAll[['Demand']])
from sklearn.externals import joblib
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['scaler.save']

In [ ]:
a = [[0.123, 0.3, 0.4445, 0.666666]]
scaler = joblib.load(scaler_filename) 
a = scaler.inverse_transform(a)

In [ ]:
scaler.inverse_transform([[0.859315634]])

array([[291.80296573]])

In [ ]:
scaler = joblib.load("scaler.save") 

In [6]:
datasetAll
su17 = datasetAll.loc[0:5663]
au17 = datasetAll.loc[5664:14495]
wi17 = datasetAll.loc[14496:23327]
sp17 = datasetAll.loc[23328:32059]
su18 = datasetAll.loc[32060:40699]
au18 = datasetAll.loc[40700:49531]
wi18 = datasetAll.loc[49532:58363]
sp18 = datasetAll.loc[58364:67095]
su19 = datasetAll.loc[67096:75735]
au19 = datasetAll.loc[75736: 84567]
wi19 = datasetAll.loc[84568: 93399]
sp19 = datasetAll.loc[93400:102131]
su20 = datasetAll.loc[102132:]

In [ ]:
datasetAll.head()

,Date,Time,Demand,Temp,isWeekend,isNotSchoolTerm,isHoliday,isNotSchoolOrHoliday,Demand_scaled,Temp_scaled,Out
0,1/1/17,0:00:00,62.861,23.8,1,1,1,1,0.185116,0.692671,0.185116
1,1/1/17,0:15:00,66.649,23.8,1,1,1,1,0.196271,0.692671,0.196271
2,1/1/17,0:30:00,73.032,23.8,1,1,1,1,0.215068,0.692671,0.215068
3,1/1/17,0:45:00,90.737,23.8,1,1,1,1,0.267207,0.692671,0.267207
4,1/1/17,1:00:00,86.345,23.3,1,1,1,1,0.254273,0.680851,0.254273


In [7]:
def split_dataset(dataset, test_fraction = 0.2):
  ## Split the dataset into train and test set ##
  train_size = int(len(dataset) * (1 - test_fraction))
  train_data, test_data = dataset.loc[:train_size], dataset.loc[train_size:]
  return train_data, test_data

def split_sequence(sequence, n_steps_in, n_steps_out):
	## Split a univariate sequence into samples ##
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import *
print(tensorflow.__version__)

2.4.1


In [8]:
n_steps_in= 30
n_steps_out = 32

whichSeason = 0 
# split dataset into train and test set
train, test = split_dataset(su17.append(su18, ignore_index=True).append(su19, ignore_index=True).append(su20, ignore_index = True), test_fraction= 0.2)

demand_scaler = MinMaxScaler()
train['Demand_scaled'] = demand_scaler.fit_transform(train[['Demand']])
test['Demand_scaled'] = demand_scaler.transform(test[['Demand']])
# split into samples
X_train, y_train = split_sequence(train['Demand_scaled'].values, n_steps_in, n_steps_out)
X_test, y_test = split_sequence(test['Demand_scaled'].values, n_steps_in, n_steps_out)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
def build_model(X_train, y_train, X_test, y_test, n_steps_in, n_steps_out, epochs, verbose): 
  ## Build an LSTM model ##
    # Define model
    model = Sequential()
    model.add(LSTM(20, activation='relu', return_sequences= 1, input_shape=(n_steps_in, 1)))
    model.add(Dropout(0.02))
    model.add(LSTM(20, return_sequences= 1))
    model.add(Dropout(0.05))
    model.add(LSTM(20, return_sequences = 0))
    
    model.add(Dense(n_steps_out)) 
    
    # Train Model
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X_train, y_train, epochs=epochs, verbose=verbose, validation_split= 0.2, shuffle = False ,callbacks = [EarlyStopping(patience = 10)])
    
    # Visualise Epoch
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()

    return model, history

model, history = build_model(X_train, y_train, X_test, y_test, n_steps_in = n_steps_in, n_steps_out = n_steps_out, epochs = 100, verbose = 2)

In [ ]:
def RMSE(model, X_test, y_test, n_steps_out, scaler):
  y_hat = scaler.inverse_transform(model.predict(X_test))
  RMSE = np.sqrt(mean_squared_error(scaler.inverse_transform(y_test.reshape(y_hat.shape[0], y_hat.shape[1])), y_hat, multioutput= 'raw_values'))
  mean_RMSE = sum(RMSE)/ n_steps_out  
  return y_hat, RMSE, mean_RMSE
y_hat_train, train_RMSE, train_mean_RMSE =RMSE(model, X_train, y_train, n_steps_out, demand_scaler)
y_hat,RMSE, mean_RMSE = RMSE(model, X_test, y_test, n_steps_out, demand_scaler)
print("Train")
print(train_RMSE, train_mean_RMSE)
print("Test")
print(RMSE, mean_RMSE)

In [ ]:
import random
# need to take into account that the in mul model
def shifting_plot(y_hat, y_test, scaler, specific_plot = -1):
  i = random.randint(0, y_hat.shape[0]-n_steps_out)
  end_index = n_steps_out
  position_start_index = 0
  
  plt.figure(figsize=(20,10))
  plt.plot(scaler.inverse_transform(y_test.reshape(y_hat.shape[0], y_hat.shape[1]))[i], label = "Truth", marker = 'x', alpha = 0.1, color = 'orange', linewidth = 5)

  if(specific_plot == -1):
    while(end_index > 0):
      plt.plot(pd.Series(data = (y_hat.reshape(y_hat.shape[0],y_hat.shape[1])[i])[0:end_index], index = np.arange(n_steps_out)[position_start_index:n_steps_out] ), marker = '4')
      i+=1
      end_index-=1
      position_start_index += 1
  else:
    plt.plot(pd.Series(data = (y_hat.reshape(y_hat.shape[0],y_hat.shape[1])[i+specific_plot])[0:end_index-specific_plot], index = np.arange(n_steps_out)[position_start_index + specific_plot:n_steps_out] ), marker = '4')  
  
  
  plt.xticks(np.arange(n_steps_out), array([str(n) for n in range(1,n_steps_out+1)]))
  plt.xlabel("nth step in the future")
  plt.legend()
  plt.ylabel("kVA")
  plt.show()

shifting_plot(y_hat, y_test, demand_scaler, -1 )
  

In [ ]:
cd Demands_Model/Armidale/

/content/gdrive/My Drive/KeyEnergy/Armidale/Demands_Model/Armidale


In [ ]:
model.save('Armidale_20210128_1230_in30out32_summer_RMSE22')

INFO:tensorflow:Assets written to: Armidale_20210128_1230_in30out32_summer_RMSE22/assets


INFO:tensorflow:Assets written to: Armidale_20210128_1230_in30out32_summer_RMSE22/assets
